In [34]:
%load_ext autoreload
%autoreload 2

import sys, os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from prompters import padding
from utils.parser_utils import get_args

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

from loss import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. u, s, v = torch.svd(param, some=False)를 수행할 때, 
## s는 행렬이 아닌 값의 형태로 나온다. 그러므로 torch.diag로 행렬화를 시켜야한다

In [35]:
random_matrix = torch.rand(3, 3)
print("원본 행렬:")
print(random_matrix)

# 특이값 분해 수행
u, s, v = torch.svd(random_matrix)

print("U 행렬:")
print(u)
print("특이값(Singular values):")
print(s)
print("V 전치 행렬:")
print(v)

원본 행렬:
tensor([[0.7771, 0.0109, 0.6626],
        [0.3043, 0.6341, 0.4025],
        [0.6220, 0.9215, 0.9744]])
U 행렬:
tensor([[-0.4585,  0.8686, -0.1879],
        [-0.4114, -0.3949, -0.8215],
        [-0.7877, -0.2994,  0.5384]])
특이값(Singular values):
tensor([1.8594, 0.6470, 0.0963])
V 전치 행렬:
tensor([[-0.5225,  0.5698, -0.6343],
        [-0.5334, -0.7988, -0.2782],
        [-0.6652,  0.1930,  0.7213]])


### SVD(Singular Value Decomposition)는 정확한 복원이 아닌 근사적인 복원이다.. 
### SVD는 정보 손실이 발생할 수 있는 근사적인 방법이기 때문이므로..

In [36]:
# 대각 성분에 특이값을 가지는 대각 행렬 S를 대각행렬로 변환
S_diag = torch.diag(s)
restored_matrix = torch.matmul(torch.matmul(u, S_diag), v)

# random_matrix와 restored_matrix가 동일한지 확인
are_equal = torch.equal(random_matrix, restored_matrix)
print("random_matrix와 restored_matrix가 동일한가요?:", are_equal)

print("="*20)

print("원본 행렬:")
print(random_matrix)

print("복원 행렬:")
print(restored_matrix)

# 복원된 행렬과 원본 행렬 간의 차이 계산
difference = torch.abs(random_matrix - restored_matrix)
print("원본 행렬과 복원된 행렬 간의 차이:")
print(difference)

random_matrix와 restored_matrix가 동일한가요?: False
원본 행렬:
tensor([[0.7771, 0.0109, 0.6626],
        [0.3043, 0.6341, 0.4025],
        [0.6220, 0.9215, 0.9744]])
복원 행렬:
tensor([[ 0.1577, -0.9382,  0.3714],
        [ 0.5885, -0.2470,  0.4992],
        [ 0.8341, -0.6698,  1.0204]])
원본 행렬과 복원된 행렬 간의 차이:
tensor([[0.6194, 0.9491, 0.2911],
        [0.2843, 0.8811, 0.0967],
        [0.2121, 1.5913, 0.0460]])


## 2. 3차원 행렬에 대한 SVD

In [37]:
# 원본 행렬 생성 (3D 텐서, shape: (2, 3, 3))
original_matrix = torch.rand(2, 3, 3)
print("원본 행렬:")
print(original_matrix)

# 각 차원에 대해 SVD 수행
U, S, Vt = torch.svd(original_matrix)

print("특이값:")
print(S)

# 대각 성분에 특이값을 가지는 대각 행렬로 변환
S_diag = torch.zeros(2, 3, 3)  # 대각 행렬 초기화
S_diag[:, :3, :3] = torch.diag_embed(S)  # 대각 성분에 특이값 할당

print("특이값을 대각행렬로 변환:")
print(S_diag)

# 원본 행렬 복원
restored_matrix = torch.matmul(torch.matmul(U, S_diag), Vt)

print("복원된 원본 행렬:")
print(restored_matrix)

# 복원된 행렬과 원본 행렬 간의 차이 계산
difference = torch.abs(original_matrix - restored_matrix)
print("원본 행렬과 복원된 행렬 간의 차이:")
print(difference)

원본 행렬:
tensor([[[0.5797, 0.4879, 0.0146],
         [0.5116, 0.4873, 0.7231],
         [0.4285, 0.9451, 0.6528]],

        [[0.5345, 0.1317, 0.9904],
         [0.0476, 0.1328, 0.6050],
         [0.2977, 0.6610, 0.3294]]])
특이값:
tensor([[1.6745, 0.4647, 0.2822],
        [1.3924, 0.5599, 0.2267]])
특이값을 대각행렬로 변환:
tensor([[[1.6745, 0.0000, 0.0000],
         [0.0000, 0.4647, 0.0000],
         [0.0000, 0.0000, 0.2822]],

        [[1.3924, 0.0000, 0.0000],
         [0.0000, 0.5599, 0.0000],
         [0.0000, 0.0000, 0.2267]]])
복원된 원본 행렬:
tensor([[[ 0.0507, -0.2062,  0.7275],
         [ 0.7133, -0.4205,  0.5801],
         [ 0.5409, -0.8377,  0.7133]],

        [[ 0.6189, -0.2519,  0.9151],
         [ 0.1037, -0.2257,  0.5694],
         [ 0.0944,  0.4117,  0.6749]]])
원본 행렬과 복원된 행렬 간의 차이:
tensor([[[0.5290, 0.6941, 0.7130],
         [0.2016, 0.9079, 0.1430],
         [0.1124, 1.7828, 0.0605]],

        [[0.0844, 0.3836, 0.0753],
         [0.0561, 0.3585, 0.0356],
         [0.2033, 0.2492, 0.3456]]]

## 2. 4차원 행렬에 대한 SVD

In [38]:
# 원본 텐서 생성 (4D 텐서, shape: (48, 48, 3, 3))
original_tensor = torch.rand(48, 48, 3, 3)
print("원본 텐서:")
print(original_tensor.shape)

# 각 차원에 대해 SVD 수행
U, S, Vt = torch.svd(original_tensor.view(-1, 3, 3))  # 텐서를 2D로 변환하여 SVD 수행

# 대각 성분에 특이값을 가지는 대각 행렬로 변환
S_diag = torch.diag_embed(S)  # 대각 성분에 특이값 할당

# 원본 텐서 복원 (4D로 변환)
restored_tensor = torch.matmul(torch.matmul(U, S_diag), Vt.transpose(1, 2)).view(48, 48, 3, 3)

print("복원된 원본 텐서:")
print(restored_tensor.shape)

# 복원된 텐서와 원본 텐서 간의 차이 계산
difference = torch.abs(original_tensor - restored_tensor)
print("원본 텐서와 복원된 텐서 간의 차이:")
print(difference.max())  # 차이의 최댓값 출력

원본 텐서:
torch.Size([48, 48, 3, 3])
복원된 원본 텐서:
torch.Size([48, 48, 3, 3])
원본 텐서와 복원된 텐서 간의 차이:
tensor(1.0133e-06)


In [39]:
print("특이값 shape : ")
print(S.shape)

print("특이값 S : ")
print(S)

print("특이값 대각성분의 shape : ")
print(S_diag.shape)

print("특이값 대각성분 : ")
print(S_diag)

특이값 shape : 
torch.Size([2304, 3])
특이값 S : 
tensor([[1.5718, 0.6653, 0.2778],
        [1.5044, 0.4394, 0.2432],
        [1.8008, 0.6610, 0.2498],
        ...,
        [1.2431, 0.4998, 0.0105],
        [1.5516, 0.3021, 0.1116],
        [1.7547, 0.4180, 0.1979]])
특이값 대각성분의 shape : 
torch.Size([2304, 3, 3])
특이값 대각성분 : 
tensor([[[1.5718, 0.0000, 0.0000],
         [0.0000, 0.6653, 0.0000],
         [0.0000, 0.0000, 0.2778]],

        [[1.5044, 0.0000, 0.0000],
         [0.0000, 0.4394, 0.0000],
         [0.0000, 0.0000, 0.2432]],

        [[1.8008, 0.0000, 0.0000],
         [0.0000, 0.6610, 0.0000],
         [0.0000, 0.0000, 0.2498]],

        ...,

        [[1.2431, 0.0000, 0.0000],
         [0.0000, 0.4998, 0.0000],
         [0.0000, 0.0000, 0.0105]],

        [[1.5516, 0.0000, 0.0000],
         [0.0000, 0.3021, 0.0000],
         [0.0000, 0.0000, 0.1116]],

        [[1.7547, 0.0000, 0.0000],
         [0.0000, 0.4180, 0.0000],
         [0.0000, 0.0000, 0.1979]]])


In [40]:
# 원본 텐서 생성 (4D 텐서, shape: (48, 48, 3, 3))
original_tensor
u, s, v = torch.svd(original_tensor, some=False)  # SVD 수행

s_diag = torch.diag_embed(s)  # 대각 성분에 특이값 할당

In [41]:
print("특이값 shape : ")
print(s.shape)

print("특이값 s : ")
print(s)

print("특이값 대각성분 : ")
print(s_diag)

특이값 shape : 
torch.Size([48, 48, 3])
특이값 s : 
tensor([[[1.5718, 0.6653, 0.2778],
         [1.5044, 0.4394, 0.2432],
         [1.8008, 0.6610, 0.2498],
         ...,
         [1.4093, 0.4884, 0.3167],
         [1.1808, 0.4806, 0.0603],
         [1.5052, 0.8864, 0.2978]],

        [[1.5492, 0.6541, 0.2755],
         [1.7355, 0.4814, 0.2145],
         [1.5273, 0.4489, 0.1835],
         ...,
         [1.4664, 0.5839, 0.1831],
         [1.7884, 0.3887, 0.1910],
         [2.2454, 0.2443, 0.1849]],

        [[2.1227, 0.2320, 0.0311],
         [1.4048, 0.2655, 0.2347],
         [1.6196, 0.7151, 0.1630],
         ...,
         [1.7020, 0.7148, 0.0684],
         [1.6474, 0.5391, 0.2984],
         [1.1734, 0.6121, 0.0225]],

        ...,

        [[1.3571, 0.1785, 0.0375],
         [1.6340, 0.8873, 0.2745],
         [1.1305, 0.3230, 0.0364],
         ...,
         [1.8168, 0.6431, 0.1445],
         [1.6824, 0.1781, 0.1058],
         [1.8063, 0.3325, 0.0180]],

        [[1.6983, 0.5236, 0.2931],
 